# ANALYSIS

In [6]:
import pandas as pd
import ast
import numpy as np

# Load the csv file

In [10]:
# df = pd.read_csv('best-books.csv',index_col='book_index')
df = pd.read_csv('best-books.csv')

# frames = [df1, df]
# result = pd.concat(frames)
# result = df1.append(df)

# filenames = ["best-books.csv", "best-books1.csv"] 
# df = pd.DataFrame()
# for filename in filenames:
#     df = df.append(pd.read_csv(filename))
df.shape

(2047, 14)

# Change the awards col into num of awards:

In [ ]:
df = df.replace({'awards': {np.nan: 0}})

d = []

for i in df.index:
    if df.loc[i, "awards"] == 0:
        d.append(0)
    else:
        d.append( len( ast.literal_eval(df.loc[i, "awards"]) ) )

df['awards length'] = d
# df['awards length'].sum()

# min max normalization

In [ ]:
df_min_max = df['avg_rating'].copy()
def min_max_normalization(df_min_max):
    df_min_max = 1 + (df_min_max - df_min_max.min()) / (df_min_max.max() - df_min_max.min()) * 9
    return round(df_min_max, 2)

minmax_norm_ratings = min_max_normalization(df_min_max)
# print(minmax_norm_ratings)

df['minmax_norm_ratings'] = minmax_norm_ratings
# print(df)

# mean normalization

In [ ]:
def mean_normalization(df_min_max):
    df_min_max = 1 + (df_min_max - df_min_max.mean()) / (df_min_max.max() - df_min_max.min()) * 9
    return round(df_min_max, 2)

mean_norm_ratings = mean_normalization(df_min_max)

df['mean_norm_ratings'] = mean_norm_ratings
# print(df)

# sort by minmax_norm_ratings Given Author

In [ ]:
def check_author(df, author):
    df = df.loc[df.author == author]
    df.sort_values(by=['minmax_norm_ratings'], ascending=[False])
    return df

# check_author(df, 'J.K. Rowling')

# sort by publishing year and get mean of minmax_norm_ratings

In [ ]:
df.groupby('original_publish_year')['minmax_norm_ratings'].mean()

# sort by author and check who has highest average rating

In [ ]:
df.groupby('author')['minmax_norm_ratings'].mean().sort_values( ascending=False)

# Sort by author and total no of awards

In [ ]:
df.groupby('author')['awards length'].sum().sort_values( ascending=False)
# df.sort_values(by=['awards length', 'author'], ascending=[False, True])

# sort by minmax_norm_ratings Given Year of publication

In [ ]:
def check_year(df, year):
    df = df.loc[df.original_publish_year == year]
    df.sort_values(by=['minmax_norm_ratings'], ascending=[False])
    return df

# check_year(df, 2005)

# check if book series are rated better than non series books + the total no of books of each type

In [ ]:
df.groupby('series')['minmax_norm_ratings'].mean()
df.groupby('series').size()

# compare no of ratings with average ratings

In [ ]:
df[ (df['num_ratings'] < df['num_ratings'].std()) & (df['avg_rating']>4) ].sort_values(by=['num_ratings'], ascending=[True])

# clean data in PLACES column

In [ ]:
df = df.replace({'places': {np.nan: '[]'}})

genres_rating = {}
genres_counts = {}

places_rating = {}
places_counts = {}

genres,places,avg_rating = df.genres, df.places, df.avg_rating

for id in range(len(avg_rating)):
    clean_genre = ast.literal_eval(genres[id])
    clean_places = ast.literal_eval(places[id])
    if len(clean_genre) != 0:
        for ng in clean_genre:
            for i in ng.split('>'):
                genr = i.strip()
                if genr in genres_rating:
                    genres_rating[genr].append(avg_rating[id])
                else:
                    genres_rating[genr] = [avg_rating[id]]
                if genr in genres_counts:
                    genres_counts[genr] += 1
                else:
                    genres_counts[genr] = 1
    #for places
    if len(clean_places) != 0:
        for cp in clean_places:
            plac = cp.strip()
            if plac in places_rating:
                places_rating[plac].append(avg_rating[id])
            else:
                places_rating[plac] = [avg_rating[id]]
            if plac in places_counts:
                places_counts[plac] += 1
            else:
                places_counts[plac] = 1

#genres
for g in genres_rating.keys():
    genres_rating[g] = np.round(np.mean(genres_rating[g]),decimals=3)

dic_genres_rating = dict(zip(genres_rating.keys(), genres_rating.values()))
df_genres_rating = pd.DataFrame.from_dict(dic_genres_rating, orient='index',\
    columns=['mean_ratings']).sort_values(by='mean_ratings',ascending=False)

dic_genres_counts = dict(zip(genres_counts.keys(), genres_counts.values()))
df_genres_counts = pd.DataFrame.from_dict(dic_genres_counts, orient='index',\
    columns=['total_counts']).sort_values(by='total_counts',ascending=False)

#places
for g in places_rating.keys():
    places_rating[g] = np.round(np.mean(places_rating[g]),decimals=3)

dic_places_rating = dict(zip(genres_rating.keys(), genres_rating.values()))
df_places_rating = pd.DataFrame.from_dict(dic_places_rating, orient='index',\
    columns=['mean_ratings']).sort_values(by='mean_ratings',ascending=False)

dic_places_counts = dict(zip(places_counts.keys(), places_counts.values()))
df_places_counts = pd.DataFrame.from_dict(dic_places_counts, orient='index',\
    columns=['total_counts']).sort_values(by='total_counts',ascending=False)

df_genres_rating.head()
df_genres_counts.head()

df_places_rating.head()
df_places_counts.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))

plt.title('', fontsize=16)
plt.xlim([-40,40])
plt.xlabel('')
plt.ylabel('')
plt.legend()
plt.savefig('.png')